In [26]:
# read gcode to get coords and interleave
# need dimensions of a slice
# 10 x 10 x 10 mm for many_layers_cube.gcode
import json
import os
import numpy as np
from PIL import Image
from PIL import ImageOps
from skimage import color as skcolor
from skimage import io as skio
from skimage import novice as sknov
from skimage import draw as skdr
import matplotlib.pyplot as plt
from scipy.misc import imresize
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.collections import LineCollection

### CONSTANTS

MATERIAL_1 = "material 1"
MATERIAL_2 = "material 2"
MATERIAL_3 = "material 3"

# noop material
MATERIAL_NOOP = "noop"

# color map for visualizing print
COLOR_MAP = {MATERIAL_1: 'r', MATERIAL_2: 'g', MATERIAL_3: 'b'}
MATERIAL_MAP = {MATERIAL_1: 0, MATERIAL_2: 1, MATERIAL_3: 2}
PSI_MAP = {MATERIAL_1: 4, MATERIAL_2: 5, MATERIAL_3: 6}
EXTRUDER_TO_PIN = {0:4, 1:5, 2:6}

###

class GCommand(object):
    """Class representing a single action of the a microvalve"""

    def __init__(self, x, y, z, material, usecs=100):
        """
        Init
        x, x location in gcode coords
        y, y location in gcode coords
        z, z location in gcode coords
        material, material indicator
        usecs, delay after movement
        """
        self.x = x
        self.y = y
        self.z = z
        self.material = material
        self.usecs = usecs
    
    def __str__(self):
        """Returns gcode representation of command"""
        if self.material == MATERIAL_1:
            return "G0 E10; G1 X{} Y{} ;material: {}\nM400 ;wait for position\nG4 P100\nM430 S{} ;send pulse\n"\
            .format(self.x, self.y, self.material, self.usecs)
        elif self.material == MATERIAL_2:
            return "G0 E20; G1 X{} Y{} ;material: {}\nM400 ;wait for position\nG4 P100\nM430 S{} ;send pulse\n"\
            .format(self.x, self.y, self.material, self.usecs)
        elif self.material == MATERIAL_3:
            return "G0 E30; G1 X{} Y{} ;material: {}\nM400 ;wait for position\nG4 P100\nM430 S{} ;send pulse\n"\
            .format(self.x, self.y, self.material, self.usecs)
        elif self.material == MATERIAL_NOOP:
            return ""

        raise ValueError("Unknown material: {}".format(self.material))

class TCommand(object):
    """Class representing a single action of the a microvalve"""

    def __init__(self, material, prev_pin=4, curr_pin=4, psi=0):
        """
        Init
        prev_material, previous material indicator
        material, material indicator
        psi, user input for pressure regulator
        """
        self.prev_pin = prev_pin
        self.curr_pin = EXTRUDER_TO_PIN[material]
        self.material = material
        self.psi = psi
    
    def __str__(self):
        """Returns gcode representation of command"""
        try:
            if self.prev_pin == -1:
                return "T{}; M42 P{} S{};\n"\
                .format(self.material,\
                        self.curr_pin,\
                        self.psi)
            return "M42 P{} S255; T{}; M42 P{} S{};\n"\
            .format(self.prev_pin,\
                    self.material,\
                    self.curr_pin,\
                    self.psi)
        except:
            raise ValueError("Unknown material: {}".format(self.material))
            
def write_gcode(layered_content, gcode_path, layer_names=None, heatbed_temp=37):
    """
    Convert list of gcommands into .gcode file. Also add start and end commands

    gcommand_layers, list of lists of GCommand objects
    gcode_path, path to write output
    layer_names, names for each layer, defaults to one index naming
    heatbed_temp, start temp UNUSED
    """
    assert heatbed_temp <= 200, "{} > max temp 200".format(heatbed_temp)

    if layer_names is not None:
        assert len(layer_names) == len(gcommand_layers), \
                "not enough names for all layers, remove layer_names to default naming"

    with open(gcode_path, 'w') as gcode_file:
        for i in range(len(layered_content)):
            gcode_file.write(layered_content[i])

def main():
    #     # user input
    width, height = 34, 34
    start_x, start_y = 90, 90
    psi = 10
    
    with open('20mm_cube.gcode') as f:
        content = f.readlines()
    layered_content = []
    layer, ctr = "", 0
    found_first_z = False
    found_second_z = False
    
    start_z, z_unit = 0, 0
    
    prev_k, prev_v = -1, -1
    
    lines = []
    for i in range(len(content)):
        if content[i] != '\r\n':
            lines.append(content[i])

#     for c in content:
#         used_line = False
#         if 'G1 Z' in c and 'G1 Z5' not in c:  
#             #'T0\r\n;layer: 117\n'
#             layer += ";layer: " + str(ctr) + "\n"
#             layered_content.append(layer)
#             layer, ctr = "", ctr+1
#             used_line = True
#             if found_first_z is False:
#                 start_z = float(c.split(" ")[1][1:])
#                 found_first_z = True
#             elif found_first_z is True and found_second_z is False:
#                 temp = float(c.split(" ")[1][1:])
#                 if temp != start_z:
#                     z_unit = temp - start_z
#                     found_second_z = True
        
#         switched = False
#         for k, v in EXTRUDER_TO_PIN.items():
#             #if "T"+str(k)+"\r\n" == c:
#             if "T"+str(k)+"\r\n" in c:
#                 tcommand = TCommand(k, prev_v, v, psi)
#                 layer += str(tcommand)
#                 switched = True
#                 prev_k = k
#                 prev_v = v
#                 used_line = True
      
#         if switched is False:
#             layer += c
#             used_line = True
#         if used_line is False:
#             print used_line
            
#     layer = ";layer: " + str(ctr) + "\n"
#     for k, v in EXTRUDER_TO_PIN.items():
#         layer += "M42 P" + str(v) +" S255;\n" 
#     layered_content.append(layer)
    
#     print (len(content), len(layered_content))
#     print layered_content
#     print (start_z, z_unit)


    # slice dimensions:
    # width, height = 10 # because gcode units are mm
    # each dot is .3 mm apart

    #gcommand_layers = convert_to_gcode(width, height, start_x, start_y, start_z, z_unit, psi, len(layered_content)-1)
    #write_gcode(layered_content, gcommand_layers, '20mm_cube_out.gcode')
    write_gcode(lines, '20mm_cube_out.gcode')
if __name__ == '__main__':
    main()


In [ ]:
# M42 P# S255 (is off)
# M42 P# S### (is on)
# T0 - P4
# T1 - P5
# T2 - P6

# now can only call T# when switch extruders

# how are we editing E again? isn't E already given to us
# T# is already mentioned for dual/multiple extruders
